# Avnatis Mapping Instructions

The following describes the process for mapping the current Avantis Classes to the new Classification system. 

## Load SPARQL data

The first step is to run SPARQL queires in protege to get the revelent metadata from the new classes. The SPARQL queires should look for classes with the specific annotation propertys:
1. is_equivalent_to_Avantis_class
2. is_equivalent_to_Avantis_category
3. is_equivalent_to_tag_code
4. is_superclass_of_Avantis_class
5. is_superclass_of_Avantis_category
6. is_superclass_of_tag_code
The following code can be modified to acomplish this task. 

```sparql
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX tw: <http://www.toronto.ca/TWONTO#>

SELECT ?SPARQL (STR(?object) as ?Avantis)
	WHERE { ?SPARQL tw:is_equivalent_to_Avantis_class ?object }
```
Once the SPARQL queires are saved, they are open in python pandas dataframes. This is done with the following code: 

In [85]:
import pandas as pd

df = pd.read_excel("SPARQL_class.xlsx", header= 1)
Class = dict(zip(df['Avantis_Class'],df['TWONTO']))
df

,TWONTO,Avantis_Class
0,channel_gate,Gate
1,instrument_air_or_pneumatic_control_panel,Air Instrument
2,UV_disinfection_assembly,Ultraviolet Disinfection
3,clarifier,Clarifier
4,controlled_door_access,Door
...,...,...
96,chiller,Chiller
97,hydraulic_system,Hydraulic System
98,filtration_tank,Biofilter
99,load_break_disconnect_switch,Load Break Switch


There is an additonal step were the two column are saved as a dictionary. A python dictionary is a simple key value pair. 

In [86]:
df = pd.read_excel("SPARQL_superclass.xlsx", header= 1)
Super_Class = dict(zip(df['Super_Class'],df['TWONTO']))

df = pd.read_excel("SPARQL_tag.xlsx", header= 1)
Tag = dict(zip(df['Tag'],df['TWONTO']))

df = pd.read_excel("SPARQL_supertag.xlsx", header= 1)
Super_Tag = dict(zip(df['Super_Tag'],df['TWONTO']))

df = pd.read_excel("SPARQL_category.xlsx", header= 1)
Category = dict(zip(df['Category'],df['TWONTO']))

df = pd.read_excel("SPARQL_supercategory.xlsx", header= 1)
Super_Category = dict(zip(df['Super_Category'],df['TWONTO']))

Tag

{'HU': 'humidier',
 'CHL': 'chlorinator_system',
 'OZ': 'ozone_generator',
 'GB': 'gearbox',
 'ALR': 'alarm_device',
 'CLR': 'clarifier',
 'AE': 'instrument_element',
 'FM': 'pressurized_sewer_segment',
 'G': 'channel_gate',
 'PDP': '600V_insulated_case_electrical_cabinet',
 'SCAM': 'security_camera',
 'AMP_x000D_\nPA': 'public_annoucement_speaker',
 'EV': 'AC_evaporator_unit',
 'D': 'dehumidifier',
 'CU': 'ac_condenser_unit',
 'PDM': 'pulsation_dampener',
 'CRN': 'crane',
 'ENG': 'engine',
 'RD': 'rupture_disc',
 'PIP': 'pipe_segment',
 'CI': 'chiller',
 'CF': 'centrifuge',
 'AHU': 'air_handler_unit',
 'SP': 'auto_sampler',
 'DEF': 'automatic_external_defibrillator',
 'ELV': 'elevator',
 'SCBA': 'self-contained_breathing_apparatus',
 'LS': 'level_switch',
 'TIT_x000D_\nTT': 'temperature_transmitter',
 'PWU': 'power_washer',
 'WIT_x000D_\nWRT_x000D_\nWSC': 'weight_scale',
 'MH': 'manhole',
 'DD': 'display_panel',
 'DR': 'controlled_door_access',
 'UV': 'UV_disinfection_assembly',
 'GA'

## SQL connection
The next step connects to the Avantis SQL server to get the list of entities

In [87]:
import pyodbc
import os
from sqlalchemy.engine import URL
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

sns.set_theme()

connect = 'DSN=Avantis6-P;UID='+ os.environ['Avantis_User'] + ';PWD=' + os.environ['Avantis_Pass']
connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connect})

engine = create_engine(connection_url)

SQL1 = """SELECT Distinct MAINTENT.id as [Entity_number],
[MAINTENT].[aenm] as [Description],
MAINTENT2.id as [Parent],
[contname] as [Category],
[entclsid] as [Class],
SUSPEND.suspoi as [Suspended]

FROM  [AvantisP].[mc].[MAINTENT]
	  Left Join [AvantisP].[mc].[ENTCLASS] on MAINTENT.entclsref_oi = ENTCLASS.entcloi
	  Left Join [AvantisP].[mc].CATVAL ON MAINTENT.cat1_oi = CATVAL.cvoi
	  Left Join MC.SUSPEND ON MAINTENT.susp_oi = SUSPEND.suspoi
	  Left Join MC.MELINK ON MAINTENT.mtnoi = MELINK.mtnchild_oi
	  Left Join MC.MAINTENT MAINTENT2 ON MELINK.mtnparn_oi = MAINTENT2.mtnoi"""


df1 = pd.read_sql(SQL1,engine)
df1 = df1[df1['Entity_number'].values != None]
df1['Tag'] = df1['Entity_number'].str.extract(r"-([a-zA-Z]+)-\d+")
df1

,Entity_number,Description,Parent,Category,Class,Suspended,Tag
0,\tFCL_ELS_CBL_001G,"Electrical Power Line,4.16KV,From BUS-00B1 to ...",FCL_ELS_4.16KV_LINES,Air Handling Unit,Electrical Power Line,NaN,NaN
1,\tFCL_ELS_CBL_002D,"Electrical Power Line,4.16KV,From BUS-00B2-A t...",FCL_ELS_4.16KV_LINES,Air Handling Unit,Electrical Power Line,NaN,NaN
2,\tFCL_ELS_CBL_002H,"Electrical Power Line,4.16KV,From BUS-00B1 to ...",FCL_ELS_4.16KV_LINES,Air Handling Unit,Electrical Power Line,NaN,NaN
3,\tFCL_ELS_CBL_002L,"Electrical Power Line,4.16KV,From BUS-00B2-A t...",FCL_ELS_4.16KV_LINES,Air Handling Unit,Electrical Power Line,NaN,NaN
4,\tFCL_ELS_CBL_003F,"Electrical Power Line,4.16KV,From BUS-00B2 to ...",FCL_ELS_4.16KV_LINES,Air Handling Unit,Electrical Power Line,NaN,NaN
...,...,...,...,...,...,...,...
147816,YX2412A,"Pump, Unwatering, Primary Tanls 3-4",TAB-PRM-P-SUMP,"Pump,Centrifugal,single-Stage",Pump,NaN,NaN
147817,Z6042S,"Extinguisher, Fire, SN#6042, Dry, Reactor Area...",TAB-SUSPENDED ENTITY,"Fire Prevention,Extinguisher",Fire Equipment,42675.0,NaN
147818,Z6042S,"Extinguisher, Fire, SN#6042, Dry, Reactor Area...",TAB-WA7-FEQ-2182S,"Fire Prevention,Extinguisher",Fire Equipment,42675.0,NaN
147819,Z73887S,"Extinguisher, Fire, SN#73887, Dry, H.P. Sub-Ba...",TAB-WA7-FEQ-2182S,"Fire Prevention,Extinguisher",Fire Equipment,42676.0,NaN


In [88]:
df1 = df1[df1['Suspended'].isna()]

In [89]:
entityMatch = pd.DataFrame()
entityMatch['Entity_number'] = df1['Entity_number'] 
entityMatch['Class_Match'] = [Class.get(x,"") for x in df1['Class']]
entityMatch['Super_Class_Match'] = [Super_Class.get(x,"") for x in df1['Class']]
entityMatch['Tag_Match'] = [Tag.get(x,"") for x in df1['Tag']]
entityMatch['Super_Tag_Match'] = [Super_Tag.get(x,"") for x in df1['Tag']]
entityMatch['Category_Match'] = [Category.get(x,"") for x in df1['Category']]
entityMatch['Super_Category_Match'] = [Super_Category.get(x,"") for x in df1['Category']]
entityMatch

,Entity_number,Class_Match,Super_Class_Match,Tag_Match,Super_Tag_Match,Category_Match,Super_Category_Match
0,\tFCL_ELS_CBL_001G,,cable_segment,,,,air_handler_unit
1,\tFCL_ELS_CBL_002D,,cable_segment,,,,air_handler_unit
2,\tFCL_ELS_CBL_002H,,cable_segment,,,,air_handler_unit
3,\tFCL_ELS_CBL_002L,,cable_segment,,,,air_handler_unit
4,\tFCL_ELS_CBL_003F,,cable_segment,,,,air_handler_unit
...,...,...,...,...,...,...,...
147813,YTF-STORE,,,,,,
147814,YTF-VSTORE,,,,,,
147815,YX2411A,pump_-_without_drive,,,,,
147816,YX2412A,pump_-_without_drive,,,,,


In [90]:
counts = [entityMatch['Class_Match'] != ""]
counts.append(counts[0] | (entityMatch['Super_Class_Match'] != ""))
counts.append(counts[1] | (entityMatch['Tag_Match'] != ""))
counts.append(counts[2] | (entityMatch['Super_Tag_Match'] != ""))
counts.append(counts[3] | (entityMatch['Category_Match'] != ""))
counts.append(counts[4] | (entityMatch['Super_Category_Match'] != ""))

for row in counts:
    print(sum(row)/len(row)*100)

78.2122503711508
79.0966269176125
83.00927477930496
83.59752885398208
86.70721389460914
87.13822773494245


In [91]:
entityMatch.groupby('Class_Match').count()['Entity_number'].sort_values(ascending=False)

Class_Match
valve                       39425
                            27297
instrument_transmitter       7903
manhole                      5899
computer                     5208
                            ...  
quencher                        3
vortex_classifier               3
UV_disinfection_assembly        2
boat                            1
network_switch                  1
Name: Entity_number, Length: 93, dtype: int64

In [92]:
Tag

{'HU': 'humidier',
 'CHL': 'chlorinator_system',
 'OZ': 'ozone_generator',
 'GB': 'gearbox',
 'ALR': 'alarm_device',
 'CLR': 'clarifier',
 'AE': 'instrument_element',
 'FM': 'pressurized_sewer_segment',
 'G': 'channel_gate',
 'PDP': '600V_insulated_case_electrical_cabinet',
 'SCAM': 'security_camera',
 'AMP_x000D_\nPA': 'public_annoucement_speaker',
 'EV': 'AC_evaporator_unit',
 'D': 'dehumidifier',
 'CU': 'ac_condenser_unit',
 'PDM': 'pulsation_dampener',
 'CRN': 'crane',
 'ENG': 'engine',
 'RD': 'rupture_disc',
 'PIP': 'pipe_segment',
 'CI': 'chiller',
 'CF': 'centrifuge',
 'AHU': 'air_handler_unit',
 'SP': 'auto_sampler',
 'DEF': 'automatic_external_defibrillator',
 'ELV': 'elevator',
 'SCBA': 'self-contained_breathing_apparatus',
 'LS': 'level_switch',
 'TIT_x000D_\nTT': 'temperature_transmitter',
 'PWU': 'power_washer',
 'WIT_x000D_\nWRT_x000D_\nWSC': 'weight_scale',
 'MH': 'manhole',
 'DD': 'display_panel',
 'DR': 'controlled_door_access',
 'UV': 'UV_disinfection_assembly',
 'GA'

In [93]:
df1[[not x for x in counts[5]]].to_csv("test.csv")

In [94]:
df1['Class'].isna().sum()
df1[[not x for x in counts[5]]][df1[[not x for x in counts[5]]]['Tag']=='PS']

,Entity_number,Description,Parent,Category,Class,Suspended,Tag
7794,COL-RBB-PS-0001,"Power Supply, DC",COL-RBB,Uninterruptible Power Supply,None,NaN,PS
7795,COL-RBB-PS-0001,"Power Supply, DC",COL-RBB-MISC,Uninterruptible Power Supply,None,NaN,PS
11872,FCL-ELS-PS-0003,Emergency Power,FCL-ELS,None,None,NaN,PS
20086,FHO-AMT-PS-0001,"Power Supply, Inverter, Weather Monitoring Sta...",FHO-ELS-PS-0004,None,Power Distribution Panel,NaN,PS
21230,FHO-DOZ-PS-1250,"Power Supply Unit, Ozone Generator OZ-1200, Oz...",FHO-DOZ-GEN,"Generator,Electricity",Switch,NaN,PS
...,...,...,...,...,...,...,...
125098,THC-ELS-PS-0003-M,"Emergency Power Equipment, Diesel & UPS, Build...",THC-ELS-PS-0003,None,None,NaN,PS
125099,THC-ELS-PS-0003-O,"Emergency Power Equipment, Diesel & UPS, Build...",THC-ELS-PS-0003,None,None,NaN,PS
144947,WTR-PKN-PS-0002,"Switch, Pressure-Low, Water Transmission To Di...",WTR-PKN-TRW,"Switch,Pressure-Low",Switch,NaN,PS
145445,WTR-PMI-PS-0002,"Switch, Pressure, Low/High, Discharge, Distric...",WTR-PMI-TRW,Switch,Switch,NaN,PS
